In [7]:
import asyncio

import sys
from typing import Annotated, Literal

# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai

import autogen
from autogen import ConversableAgent
config_list = azureopenai.config_list


def get_market_news(ind, ind_upper):
    import json

    import requests

    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=MSFT&sort=LATEST&limit=5&apikey=demo'
    r = requests.get(url)
    data = r.json()
   
    
    feeds = data["feed"][ind:ind_upper]
    feeds_summary = "\n".join(
        [
            f"News summary: {f['title']}. {f['summary']} overall_sentiment_score: {f['overall_sentiment_score']}"
            for f in feeds
        ]
    )
    return feeds_summary


data = asyncio.Future()


async def add_stock_price_data():
    # simulating the data stream
    for i in range(0, 5, 1):
        latest_news = get_market_news(i, i + 1)
        if data.done():
            data.result().append(latest_news)
        else:
            data.set_result([latest_news])
        # print(data.result())
        await asyncio.sleep(5)


data_task = asyncio.create_task(add_stock_price_data())

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "timeout": 600,
        "cache_seed": 41,
        "config_list": config_list,
        "temperature": 0,
    },
    system_message="You are a financial expert.",
)
# create a UserProxyAgent instance named "user"
user_proxy = autogen.UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config=False,
    default_auto_reply=None,
)


async def add_data_reply(recipient, messages, sender, config):
    await asyncio.sleep(0.1)
    data = config["news_stream"]
    if data.done():
        result = data.result()
        if result:
            news_str = "\n".join(result)
            result.clear()
            return (
                True,
                f"Just got some latest market news. Merge your new suggestion with previous ones.\n{news_str}",
            )
        return False, None


user_proxy.register_reply(autogen.AssistantAgent, add_data_reply, position=2, config={"news_stream": data})
#"Give me investment suggestion in 3 bullet points.
await user_proxy.a_initiate_chat(  # noqa: F704
    assistant,
    message="""Give me top 3 listed stocks in india to invest in""",
)
while not data_task.done() and not data_task.cancelled():
    reply = await user_proxy.a_generate_reply(sender=assistant)  # noqa: F704
    if reply is not None:
        await user_proxy.a_send(reply, assistant)  # noqa: F704

Task exception was never retrieved
future: <Task finished name='Task-16' coro=<add_stock_price_data() done, defined at C:\Users\rakhandu\AppData\Local\Temp\ipykernel_16228\912086612.py:41> exception=FileNotFoundError(2, 'No such file or directory')>
Traceback (most recent call last):
  File "C:\Users\rakhandu\AppData\Local\Temp\ipykernel_16228\912086612.py", line 44, in add_stock_price_data
    latest_news = get_market_news(i, i + 1)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rakhandu\AppData\Local\Temp\ipykernel_16228\912086612.py", line 24, in get_market_news
    with open('market_news_local.json', 'r') as file:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\proddev\research\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 324, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'market_news_local.json'


user (to assistant):

Give me top 3 listed stocks in india to invest in

--------------------------------------------------------------------------------
assistant (to user):

Investing in stocks requires careful consideration of various factors, including the company's financial health, market position, growth potential, and overall economic conditions. As of my last update in October 2023, I can provide some general guidance, but it's crucial to conduct your own research or consult with a financial advisor before making any investment decisions. Here are three well-regarded stocks in India that have historically been considered strong investments:

1. **Reliance Industries Limited (RIL)**
   - **Sector:** Conglomerate (Energy, Retail, Telecommunications)
   - **Overview:** Reliance Industries is one of the largest companies in India, with a diverse portfolio that includes petrochemicals, refining, oil, telecommunications (Jio), and retail. The company has shown consistent growth and 